# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
# print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

# write into csv file event_datafile_new
with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    
    # create column headers
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    
    # for every row in combined data set
    for row in full_data_rows_list:
        
        # skip over rows where the first field, artist, is blank 
        if (row[0] == ''):
            continue
            
        # write data into new csv file using their index
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Create a NoSQL keyspace to answer the following three requests:
#### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


#### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

#### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#### For context, the newly created <font color=red>**event_datafile_new.csv**</font> contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace 
session.execute("""CREATE KEYSPACE IF NOT EXISTS sparktify
                   WITH REPLICATION = 
                   {'class' : 'SimpleStrategy', 'replication_factor' : 1}""")

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
session.set_keyspace('sparktify')

### Create tables and queries to align with each requst stated above. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

#### Request 1: Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [8]:
# Query description: Pull song details heard for session ID 338 and item in session 4
# Query design: select artist, song, length from song_heard_during_session where \ 
# sessionId = 338 and itemInSession = 4

# Drop table placeholder
# session.execute("DROP TABLE IF EXISTS song_heard_during_item_in_session")

# Create table
create_table1 = "CREATE TABLE IF NOT EXISTS song_heard_during_item_in_session"

# Define columns to include and partition columns
# Partition columns: sessionid and iteminsession; Reasoning: both fields used in where clause and make the record unique
create_table1 += """(sessionId int,
                    itemInSession int,
                    artist varchar,
                    song varchar,
                    length decimal,
                    PRIMARY KEY (sessionId, itemInSession))"""

# Execute create table statement
session.execute(create_table1)

#### Read in <font color=red>**event_datafile_new.csv**</font>

In [9]:
# Set file variable to event_datafile_new.csv
file = 'event_datafile_new.csv'

# Read in file
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    # For each row in csv file
    for line in csvreader:
        
        # Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_heard_during_item_in_session "
        query += "(sessionId, itemInSession, artist, song, length) "
        query += "VALUES(%s,%s,%s,%s,%s)"
        
        # Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
# Test query design to verify the data was entered into the table \
# and set object to variable data1
data1 = session.execute("""select 
                               artist, 
                               song, 
                               length 
                           from song_heard_during_item_in_session 
                           where sessionId = 338 
                               and itemInSession = 4""")
    
# initialize empty list under output variable
output = []

# For every row in data1
for row in data1:
    
    # Place row of data into list under the variable row_list
    row_list = [row.artist, row.song, row.length]
    
    # Append row list to output
    output.append(row_list)

# Define column labels
column_labels = (['artist', 'song', 'length'])

# Create dataframe utilizing output and column labels
df1 = pd.DataFrame(output, columns = column_labels)

# Print dataframe
print(df1)

      artist                             song    length
0  Faithless  Music Matters (Mark Knight Dub)  495.3073


#### Request 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [11]:
# Query description: Pull user name and song details heard for user ID 10 and session id 182 \
# in the order the songs were played
# Query design: select artist, song, firstname, lastname from song_heard_during_user_session \
# where userid = 10 and sessionid = 182

# Drop table placeholder
# session.execute("DROP TABLE IF EXISTS song_heard_during_user_session")

# Create table
create_table2 = "CREATE TABLE IF NOT EXISTS song_heard_during_user_session"

# Partition columns: userid and sessionid; Reasoning: fields used in where clause
# Cluster column: iteminsession; Reasoning: to sort on iteminsession and it makes the record unique
create_table2 += """(userid int,
                    sessionid int,
                    iteminsession int,
                    artist varchar,
                    song varchar,
                    firstname varchar,
                    lastname varchar,
                    PRIMARY KEY ((userid, sessionid), iteminsession))"""

# Execute create table statement
session.execute(create_table2)         

#### Read in <font color=red>**event_datafile_new.csv**</font>

In [12]:
# Read in csv
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    # For each row in csv file
    for line in csvreader:
        
        # Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_heard_during_user_session "
        query += "(userid, sessionId, iteminsession, artist, song, firstname, lastname) "
        query += "VALUES(%s,%s,%s,%s,%s,%s,%s)"
        
        # Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

#### Do a SELECT to verify that the data have been inserted into each table

In [13]:
# Test query design to verify the data was entered into the table \
# and set object to variable data2
data2 = session.execute("""select
                               artist, 
                               song, 
                               firstname, 
                               lastname 
                            from song_heard_during_user_session 
                            where userid = 10 
                                and sessionid = 182""")

# initialize empty list under output variable
output = []

# For every row in data2
for row in data2:
    
    # Place row of data into list under the variable row_list
    row_list = [row.artist, row.song, row.firstname, row.lastname]
    
    # Append row list to output
    output.append(row_list)

# Define column labels
column_labels = (['artist', 'song', 'firstname', 'lastname'])

# Create dataframe utilizing output and column labels
df2 = pd.DataFrame(output, columns = column_labels)

# Print dataframe
print(df2)

              artist                                               song  \
0   Down To The Bone                                 Keep On Keepin' On   
1       Three Drives                                        Greece 2000   
2  Sebastien Tellier                                          Kilometer   
3      Lonnie Gordon  Catch You Baby (Steve Pitron & Max Sanna Radio...   

  firstname lastname  
0    Sylvie     Cruz  
1    Sylvie     Cruz  
2    Sylvie     Cruz  
3    Sylvie     Cruz  


### Request 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [14]:
# Query description: Pull user name(s) for anyone who listen to the song 'All Hands Against His Own'
# Query design: select firstname, lastname from user_who_heard_song where song = 'All Hands Against His Own'

# Drop table placeholder
# session.execute("DROP TABLE IF EXISTS user_who_heard_song")

# Create table
create_table3 = "CREATE TABLE IF NOT EXISTS user_who_heard_song"

# Partition column: song; Reasoning: fields used in where clause
# Cluster column: userid; Reasoning: to make the record unique
create_table3 += """(song varchar,
                     userid int,
                     firstname varchar,
                     lastname varchar,
                     PRIMARY KEY (song, userid))"""

# Execute create table statement
session.execute(create_table3)           

#### Read in <font color=red>**event_datafile_new.csv**</font>

In [15]:
# Read in csv
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    # For each row in csv file
    for line in csvreader:
        
        # Assign the INSERT statements into the `query` variable
        query = "INSERT INTO user_who_heard_song "
        query += "(song, userid, firstname, lastname) "
        query += "VALUES(%s,%s,%s,%s)"
        
        # Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

#### Do a SELECT to verify that the data have been inserted into each table

In [16]:
# Test query design to verify the data was entered into the table \
# and set object to variable data3
data3 = session.execute("""select 
                               firstname, 
                               lastname 
                            from user_who_heard_song
                            where song = 'All Hands Against His Own'""")

# initialize empty list under output variable
output = []

# For every row in data3
for row in data3:
    
    # Place row of data into list under the variable row_list
    row_list = [row.firstname, row.lastname]
    
    # Append row list to output
    output.append(row_list)

# Define column labels
column_labels = (['firstname', 'lastname'])

# Create dataframe utilizing output and column labels
df3 = pd.DataFrame(output, columns = column_labels)

# Print dataframe
print(df3)

    firstname lastname
0  Jacqueline    Lynch
1       Tegan   Levine
2        Sara  Johnson


### Drop the tables before closing out the sessions

In [17]:
# Drop the table before closing out the sessions
session.execute("DROP TABLE IF EXISTS song_heard_during_item_in_session")
session.execute("DROP TABLE IF EXISTS song_heard_during_user_session")
session.execute("DROP TABLE IF EXISTS user_who_heard_song")

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()